<a href="https://colab.research.google.com/github/Sarvesh1814/HateXplain/blob/Sarvesh/BERT_Model(Failed_Attempt).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install transformers -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 13.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Reproducibility/Sample Model/HateXplain2.csv")

In [ ]:
data = data[['post_tokens',	'Target_cat',	'Label_cat',	'final_rationales']]
data['post_tokens'] = data['post_tokens'].apply(lambda x: eval(x))
for i in range(len(data)):
  
  sentence =""
  for j in (data['post_tokens'].iloc[i]):
    
    sentence += j +" "
    
  data['post_tokens'].iloc[i] = sentence

In [ ]:
from sklearn.model_selection import train_test_split
X = data["post_tokens"]
Y= data["Label_cat"]
x_train , x_test , y_train, y_test = train_test_split(X,Y,test_size=0.2)


In [ ]:
data.to_csv("training.csv")

In [ ]:
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification

# Define the model hyperparameters
model_name = 'bert-base-uncased'
max_length = 128
batch_size = 32
num_epochs = 3
learning_rate = 2e-5

# Load the CSV file into a Pandas dataframe


# Define the label mapping
label_map = {'hatespeech': 0, 'normal': 1, 'offensive': 2}

# Define the tokenizer
tokenizer = BertTokenizer.from_pretrained(model_name)

# Tokenize and encode the data
tokenized = data["post_tokens"].apply(lambda x: tokenizer.encode_plus(x,
                                                              add_special_tokens=True,
                                                              max_length=max_length,
                                                              truncation=True,
                                                              padding='max_length',
                                                              return_attention_mask=True,
                                                              return_token_type_ids=False))

# Convert the label strings to integers
labels = data["Label_cat"].apply(lambda x: label_map[x])

dataset = []
for i in range(len(tokenized)):
    input_ids = tokenized[i]['input_ids']
    attention_mask = tokenized[i]['attention_mask']
    label = labels[i]
    dataset.append((input_ids, attention_mask, label))
dataset = tf.data.Dataset.from_generator(lambda: dataset, output_types=(tf.int32, tf.int32, tf.int32)).batch(batch_size)
# Load the pre-trained BERT model
model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=3)

# Define the optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Train the model
model.fit(dataset, epochs=num_epochs)

# Make predictions on a single sentence
def predict_sentence(sentence):
    input_ids = tokenizer.encode_plus(sentence,
                                       add_special_tokens=True,
                                       max_length=max_length,
                                       truncation=True,
                                       padding='max_length',
                                       return_attention_mask=True,
                                       return_token_type_ids=False,
                                       return_tensors='tf')['input_ids']
    attention_mask = tokenizer.encode_plus(sentence,
                                            add_special_tokens=True,
                                            max_length=max_length,
                                            truncation=True,
                                            padding='max_length',
                                            return_attention_mask=True,
                                            return_token_type_ids=False,
                                            return_tensors='tf')['attention_mask']
    logits = model.predict((input_ids, attention_mask))[0]
    predicted_label = tf.argmax(logits, axis=1).eval()[0]
    return list(label_map.keys())[list(label_map.values()).index(predicted_label)]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3


OperatorNotAllowedInGraphError: ignored

In [ ]:

# Example usage
sentence = "This is a normal tweet."
predicted_label = predict_sentence(sentence)
print(f"Sentence: {sentence}")
print(f"Predicted label: {predicted_label}")


In [ ]:
dataset

<_BatchDataset element_spec=(TensorSpec(shape=<unknown>, dtype=tf.int32, name=None), TensorSpec(shape=<unknown>, dtype=tf.int32, name=None), TensorSpec(shape=<unknown>, dtype=tf.int32, name=None))>